In [1]:
import math
import time

import PIL.Image
import ipywidgets as widgets
import torch
import torch.nn.functional as functional
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import traitlets
from IPython.display import display
import cv2

from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot


In [2]:
#angle, angle_last, stop_counter = 0.0, 0.0, 0
#stop_counter_limit = 10
#x, y = 0.0, 0.0
#frame_counter = 0

speed_default = 0.15
speed_prev_left = 0.15
speed_prev_right = 0.15
threshold_probability = 0.8 # at least 80% certainty required for flag
threshold_frames = 4 # at least 4 times in row 0.8 required for flag to be set

#import zope.interface
import subprocess

'''####################################### INTERFACE ###########################################'''

class IObserver():#zope.interface.Interface):
    def update(self):
        pass

'''####################################### INTERFACE ###########################################'''

class IObservable():#zope.interface.Interface):
    _observers = [] #indicating arrays with _
    def attach(self, observer):
        if not observer in self._observers:
            self._observers.append(observer)
    def detatch(self, observer):
        if observer in self._observers:
            self._observers.remove(observer)
    #def notify(self):
        #pass
        #for observer in self._observers:
         #   observer.update()
        #customRobot wird mit Parametern benachrichtigt (einziger Observer)

'''########################################## CUSTOMROBOT ########################################'''

class CustomRobot(Robot, IObserver):
    _ourObjects = []
    angle_last = 0.0
    model_road = 0
    image_preproc = 0
    steer_gain = 0.04
    steer_kd_gain = 0
    steer_bias = 0

    def setOurObjects(self, _ourObjects):
        self._ourObjects = _ourObjects
    def doYourThing(self):
        ###customRobot._ourObjects ==[O F L N S K]
        self._ourObjects[2].reactTo()
        self._ourObjects[5].reactTo()
        self._ourObjects[0].reactTo()
        if not self._ourObjects[0].is_present or self._ourObjects[0].was_present:
            self._ourObjects[1].reactTo()
            if not self._ourObjects[1].was_present:
                self._ourObjects[4].reactTo()
                if not self._ourObjects[4].was_present:
                     self.set_road_direction(self.model_road, self.image_preproc)
                     if not self._ourObjects[4].is_present:
                         self._ourObjects[3].reactTo()

        #Alternative: reactTo() soll returnen was passiert ist
        # --> 0 = nichts, 1 = stehen (bleiben), 3 = verzögern, 4 = beschleunigen, 5 = drehen
        # Bsp.: Sieht Stopschild --> hat nichts gemacht 0, sieht Stopschild nicht mehr --> ist stehen (geblieben) 1

    def update(self, *_probabilities): # ACHTUNG: setzt voraus, dass _probabilities und _ourObjects gleiche Reihenfolge haben !!!
        i = 0
        for ourObject in self._ourObjects:
            ourObject.setPresence(_probabilities[i])
            i+=1
        self.doYourThing()

    #@todo speeds mit limits vereinbaren
    def set_road_direction(self, model_road, image):
        model_xy = model_road(image).detach().float().cpu().numpy().flatten()
        model_x = model_xy[0]
        model_y = (0.5 - model_xy[1]) / 2.0
        angle, left, right = self.calculate_speed(self.angle_last, model_x, model_y)
        self.angle_last = angle
        self.left_motor.value = left
        self.right_motor.value = right

    #@todo steering gain und steering kd müssen per slider einstellbar sein
    #@todo Datei auf max_speed umstellen und calculate_speed() anpassen
    def calculate_speed(self, last_a: float, x_in: float, y_in: float) -> (float, float, float):
        global a_slider
        a = math.atan2(x_in, y_in)
        pid = a * self.steer_gain + (a - last_a) * self.steer_kd_gain
        steer_val = pid + self.steer_bias
        speed_control = (super.right_motor.value + super.left_motor.value) / 2 # speed control auf wish bestellt
        left = max(min(speed_control + steer_val, 1.0), 0.0)
        right = max(min(speed_control - steer_val, 1.0), 0.0)
        return a, left, right

    def setRoadModel(self, model_road):
        self.model_road = model_road

    def giveProcImage(self, image_preproc):
        self.image_preproc = image_preproc

    def set_motor_speeds(self, l, r):
        super.left_motor.value = l
        super.right_motor.value = r

'''####################################### MODEL ###########################################'''
        ### PREFERRED METHOD ###
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

'''####################################### SCOUT ###########################################'''

class Scout(IObservable):
    _models = []
    yolo_model = 0
    device = torch.device('cuda')
    mean = torch.Tensor([0.485, 0.456, 0.406]).half().float().to(device)
    std = torch.Tensor([0.229, 0.224, 0.225]).half().float().to(device)
    normalize = torchvision.transforms.Normalize(mean, std)
    camera = Camera.instance(width=224, height=224, fps=10)

    def __init__(self, customRobot):
        self.attach(customRobot)

    def createUIelements(self):
        pass

    def initializeModelsAndDevice(self):

        FILE = "model_signs_kreuzung.pt"
        model_signs = Model(n_input_features=5)
        torch.save(model_signs.state_dict(), FILE)
        model_signs.load_state_dict(torch.load(FILE))
        model_signs.eval()
        model_signs = model_signs.to(self.device)
        self.yolo_model = model_signs

        model_collision = torchvision.models.alexnet(pretrained=False)
        model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, 2)
        model_collision.load_state_dict(torch.load('model_collavoid.pth'))
        model_collision = model_collision.to(self.device)
        self._models.append(model_collision)

        model_road = torchvision.models.resnet18(pretrained=False)
        model_road.fc = torch.nn.Linear(512, 2)
        model_road.load_state_dict(torch.load('model_road_following.pth'))
        model_road = model_road.to(self.device)
        self._models.append(model_road)
        self._observers[0].setRoadModel(model_road)

        ''''
        model_collision = torchvision.models.alexnet(pretrained=False)
        model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, 2)
        model_collision.load_state_dict(torch.load('/model_collavoid.pth'))

        model_road = torchvision.models.resnet18(pretrained=False)
        model_road.fc = torch.nn.Linear(512, 2)
        model_road.load_state_dict(torch.load('/model_road_following.pth'))

        model_road = model_road.to(self.device)
        model_collision = model_collision.to(self.device)

        model_signs =  torch.hub.load('./', 'custom', path='/model_signs_kreuzung.pt', source='local')
        #model_signs = model_signs.to(self.device)

        self._models.append(model_collision, model_road)
        self._observers[0].setRoadModel(model_road)
        ''''

    def tookNewFrameProcessNotify(self, change):
        image = self.change['new']
        image_preproc = self.preprocess(image).to(self.device)
        prob_blocked = self.get_collision_chance(image_preproc)

        self.observers[0].giveProcImage(image_preproc)

        bashCommand = "python3 my-detect.py --img 224 --conf 0.4 --source image --weights 'model_signs_kreuzung.pt'"
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

        prob_sign_verbot, prob_sign_limit, prob_sign_nolimit, prob_sign_stop, prob_kreuzung = self.filterDataFrame(output)

        self._observers[0].update(prob_blocked, prob_sign_verbot, prob_sign_limit, prob_sign_nolimit, prob_sign_stop, prob_kreuzung)
        #updated nur customRobot

    #@todo NICHT ALLES IN JEDEM FRAME
    #@todo frames statt timer 

    def filterDataFrame(self, output):
        df = output.pandas().xyxy[0]
        df.sort_values(['class', 'ymax'], ascending=[True, False], inplace = True)
        df_clean = df

        _rowsToBeDeleted = []
        index_rowsToBeDeleted = 0

        ### Wenn eine Klasse mehrmals vorkommt, bleibt nur die mit dem höchsten ymax übrig
        tmp_row = 0
        for index, row in df.iterrows():
            if index > 0:
                print("row['class'] ", row['class'], "df.at[index-1, 'class'] ",df.at[index-1, 'class'])
                if row['class'] == tmp_row['class']:
                    _rowsToBeDeleted.append(index)
                    index_rowsToBeDeleted += 1
            tmp_row = row

        for i in range(len(_rowsToBeDeleted)):
            df_clean = df_clean.drop(_rowsToBeDeleted[i])
            for k in range(len(_rowsToBeDeleted)):
                _rowsToBeDeleted[k] -= 1

        prob_sign_forbidden = df_clean[df_clean["class"] == "0"]["confidence"].values[0]
        prob_sign_limit = df_clean[df_clean["class"] == "1"]["confidence"].values[0]
        prob_sign_nolimit = df_clean[df_clean["class"] == "2"]["confidence"].values[0]
        prob_sign_stop = df_clean[df_clean["class"] == "3"]["confidence"].values[0]
        prob_kreuzung = df_clean[df_clean["class"] == "4"]["confidence"].values[0]

        return prob_sign_forbidden, prob_sign_limit, prob_sign_nolimit, prob_sign_stop, prob_kreuzung


    def preprocess(self, image):
        image = PIL.Image.fromarray(image)
        image = transforms.functional.to_tensor(image).to(self.device)
        image.sub_(self.mean[:, None, None]).div_(self.std[:, None, None])
        return image[None, ...]

    def preprocess_2(self, image):
        x_2 = image
        x_2 = cv2.cvtColor(x_2, cv2.COLOR_BGR2RGB)
        x_2 = x_2.transpose((2, 0, 1))
        x_2 = torch.from_numpy(x_2).float()
        x_2 = self.normalize(x_2)
        x_2 = x_2.to(self.device)
        x_2 = x_2[None, ...]
        return x_2

    def get_collision_chance(self, image) -> float:
        collision = self._models[0](image)
        collision_softmax = functional.softmax(collision, dim=1).flatten()
        return float(collision_softmax[0])

    def get_kreuzung_chance(self, image) -> float:
        kreuzung = self._models[2](image)
        kreuzung_softmax = functional.softmax(kreuzung, dim=1).flatten()
        return float(kreuzung_softmax[0])

    def get_road_direction(self,image) -> (float, float):
        model_xy = self._models[3](image).detach().float().cpu().numpy().flatten()
        model_x = model_xy[0]
        model_y = (0.5 - model_xy[1]) / 2.0
        return model_x, model_y

'''########################################## OUROBJECT ########################################'''

class OurObject:
    frames_saw_in_row = 0
    frames_not_saw_in_row = 0 # Anzahl aufeinander folgender Frames in der ein is_present Objekt nicht mehr sichtbar ist
    is_present = False
    was_present = False

    xmin = 0
    ymin = 0
    xmax = 0
    ymax = 0

    # Beispiel: threshold_frames ist 4, threshold_probability ist 0.8
    # --> wenn 4 Frames hintereinander Stop-Schild zu 80% gesehen wurde, obj_sign_Stop.is_present auf True setzen
    # ansonsten counter reseten und not_saw counter erhöhen und is_present auf False setzen wenn not_saw counter 4 Frames erreicht.
    # was_present wird erst True, wenn davor is_present galt und jetzt nicht mehr
    # was_present wird in der jeweiligen reactTo() Methode auf False zurückgesetzt, wenn die entsprechend letzte Handlung durchgeführt wird.
    def setPresence(self, probability_present): # zum Beispiel prob_Kreuzung oder prob_Verbot oder ...
        if probability_present > threshold_probability:
            self.frames_saw_in_row += 1
            self.frames_not_saw_in_row = 0
            self.is_present = self.frames_saw_in_row > threshold_frames
        else:
            self.frames_saw_in_row = 0
            if self.is_present:
                self.frames_not_saw_in_row += 1
                if self.frames_not_saw_in_row > threshold_frames:
                    self.was_present = True
                    self.is_present = False

    def setCoordinates(self, xmin, ymin, xmax, ymax):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax


'''####################################### KREUZUNG ###########################################'''

class Kreuzung(OurObject):
    def reactTo(self):
        if self.is_present:
            customRobot.set_motor_speeds(0.11, 0.11)
        else:
            customRobot.set_motor_speeds(0.15, 0.15)
            self.was_present = False

'''####################################### SIGNNOLIMIT ###########################################'''

class SignNolimit(OurObject):
    def reactTo(self):
        if self.is_present:
            customRobot.set_motor_speeds(0.15, 0.15)
        else:
            self.was_present = False

'''####################################### SIGNVERBOT ###########################################'''

class SignVerbot(OurObject):
    phase = 0

    def __init__(self, objKreuzung):
        self.objKreuzung = objKreuzung

    def tryRight(self):
        customRobot.right(0.3)
        time.sleep(0.5)
        self.phase = 1

    def tryLeft(self):
        customRobot.left(0.3)
        time.sleep(1)
        self.phase = 2

    def goBackWhereYouCameFrom(self):
        customRobot.left(0.3)
        time.sleep(0.5)
        customRobot.set_motor_speeds(0.11, 0.11)
        self.phase = 0
        self.was_present = False

    def driveInCurrentDirection(self):
        customRobot.set_motor_speeds(0.11, 0.11)
        self.phase = 0
        self.was_present = False

    def reactTo(self):
        if self.phase == 0:
            if self.is_present:
                self.tryRight()
        elif self.phase == 1:
            if not self.objKreuzung.is_present:
                self.tryLeft()
            else:
                self.driveInCurrentDirection()
        elif self.phase == 2:
            if not self.objKreuzung.is_present:
                self.goBackWhereYouCameFrom()
            else:
                self.driveInCurrentDirection()

'''####################################### SIGNLIMIT ###########################################'''

class SignLimit(OurObject):
    def __init__(self, objNolimit):
        self.objKreuzung = objNolimit

    def reactTo(self):
        if self.is_present:
            customRobot.set_motor_speeds(0.12, 0.12)
        elif self.objNolimit.is_present:
            self.was_present = False

'''####################################### SIGNSTOP ###########################################'''

class SignStop(OurObject):
    def reactTo(self):
        if self.was_present:
            customRobot.stop()
            time.sleep(3)
            customRobot.set_motor_speeds(0.11, 0.11)
            self.was_present = False

'''####################################### OBSTACLE ###########################################'''

class Obstacle(OurObject):
    phase = 0
    timer = 0
    patience = 5 #seconds

    def goOn(self):
        customRobot.set_motor_speeds(0.11, 0.11)
        self.was_present = False
        self.phase = 0

    def stayAndWait(self):
        customRobot.stop()
        self.timer = time.time()
        self.phase = 1

    def turnBack(self): # kurz rückwärts, 180° Wende, weiterfahren
        customRobot.backward(0.3)
        time.sleep(0.5) #@todo timer Rechnungen statt sleep
        customRobot.stop()
        customRobot.left(0.3)
        time.sleep(1)
        customRobot.stop()
        customRobot.set_motor_speeds(0.11, 0.11)
        self.phase = 0
        self.was_present = False

    def reactTo(self):
        if self.is_present:
            if self.phase == 0:
                self.stayAndWait()
            elif self.phase == 1:
                if not self.is_present:
                    self.goOn()
                elif time.time() - self.timer > self.patience:
                    self.turnBack() # @todo, soll nur 180° Wende machen, wenn es links und rechts keinen Ausweg gibt


RuntimeError: Could not initialize camera.  Please see error trace.

In [4]:
scout.camera.stop()

NameError: name 'camera' is not defined

In [3]:
signStop = SignStop()
kreuzung = Kreuzung()
signNolimit = SignNolimit()
signLimit = SignLimit(signNolimit)
signVerbot = SignVerbot(kreuzung)
obstacle = Obstacle()

customRobot = CustomRobot()
### O F L N S K ###
customRobot.setOurObjects([obstacle, signVerbot, signLimit, signNolimit, signStop, kreuzung])#signVerbot, kreuzung

scout = Scout(customRobot)
scout.initializeModelsAndDevice()
scout.tookNewFrameProcessNotify({'new': scout.camera.value})
#@todo: x,y der Objekte --> speedlimits spät aufheben wenn kreuzung y 230 erreicht, --> verbot umdrehen etc

FileNotFoundError: [Errno 2] No such file or directory: './hubconf.py'

In [ ]:

def start_stop_toggle(obj):
    if obj["new"]:
        toggle_exec_btn.button_style = 'warning'
        toggle_exec_btn.icon = 'warning'
        toggle_exec_btn.description = "Stop"
        scout.camera.observe(scout.tookNewFrameProcessNotify, names='value')
    else:
        toggle_exec_btn.button_style = 'success'
        toggle_exec_btn.icon = 'check'
        toggle_exec_btn.description = 'Start'
        scout.camera.unobserve(scout.tookNewFrameProcessNotify, names='value')
        time.sleep(0.1)
        customRobot.stop()


In [3]:

toggle_exec_btn.observe(start_stop_toggle, 'value')

''' -------------------------------------------------------------------------------------------------------- '''

IndentationError: expected an indented block (2278962894.py, line 72)

In [ ]:

    '''
    Verbale Beschreibung, was das Programm tun soll:

    Wenn der Roboter blockiert ist und die Flag noch nicht gesetzt wurde,
    Flag setzen und schrittweise abbremsen.
    Im Stillstand bis zu 20 Frames warten, danach Rückwärtsgang einlegen.

    Flag bei Kreuzung setzen und langsamer werden, nach Kreuzung schneller werden (unideal bei Kurven aber egal)

    Bei Speedlimits ggf. langsamer werden. Speedlimits gelten bis zur nächsten Kreuzung.

    Bei Stopschildern Flag setzen und anhalten, wenn es nicht mehr sichtbar ist, 3s warten. (erfordert, Stopschild an gute Position zu setzen)

    Bei Durchfahrtverbotsschild 90° im Uhrzeigersinn (rechts) drehen. Bei Kreuzung weiterfahren,
    ansonsten 180° gegen den Uhrzeigersinn drehen. Bei Kreuzung weiterfahren,
    ansonsten 90° gegen den Uhrzeigersinn drehen und wieder zurück fahren

    Im Falle, dass der Roboter nicht im Rückwärtsgang ist oder sich bei Durchfahrtverbot dreht oder warten muss, den Road-Following Koordinaten (PoI) folgen

    Flags werden entsprechend wieder zurückgesetzt (hoffentlich)
    '''

    '''
    if frame_counter % camera.fps == 0:
        frame_counter = 0

        snapshot = image.copy()
        snapshot = cv2.circle(snapshot, (int(x * snapshot.width), int(y * snapshot.height)), 8, (0, 255, 0), 3)
        image_vis_widget.value = bgr8_to_jpeg(snapshot)
    frame_counter += 1
    '''

